In [2]:
import pandas as pd
import geopandas as gpd
import json
from shapely.geometry import Point, LineString

In [33]:
# data_frames = []

# for i in range(1, 16):
#     file_name = f"../res/result{i}.csv"
#     df = pd.read_csv(file_name)
#     data_frames.append(df)

# # 使用 concat 函数将所有 DataFrame 合并为一个
# result = pd.concat(data_frames)

# result.to_csv("../res/sample_taxi_region.csv")
df_order = pd.read_csv("../res/sample_taxi_region.csv")

In [34]:
# 按照出租车编号对数据进行分组
df_grouped = df_order.groupby('taxi_id')

In [35]:
# from datetime import datetime
# # 定义一个将时间字符串转换为datetime类型的函数
# def convert_time(time_string):
#     return datetime.strptime(time_string, '%H:%M:%S')

# # 使用apply行向量函数将整个时间列转换为datetime类型
# df_grouped['time'] = df_grouped['time'].apply(lambda x: convert_time(x))
# print(df_grouped.head(1000))
print(type(df_grouped))
df_grouped_slice = df_grouped.head(1000)
print(type(df_grouped_slice))

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>
<class 'pandas.core.frame.DataFrame'>


In [32]:
# 分别处理每个出租车的订单
orders = []
df_sorted = df_grouped.apply(lambda x: x.sort_values('time'))
# df_sorted.to_csv('../res/sorted.csv')
order_started = False
order_start_lat = None
order_start_lon = None
order_end_lat = None
order_end_lon = None
order_start_district = None
order_start_block = None
order_end_district = None
order_end_block = None
for _, row in df_sorted.iterrows():
    if row['is_passenger'] == 1 and not order_started:
        # 如果订单开始，记录起始点位置和时间
        order_start_lat = row['lat']
        order_start_lon = row['lon']
        order_start_time = row['time']
        order_start_district = row['region']
        order_start_block = row['block_id']
        order_started = True
    elif row['is_passenger'] == 0 and order_started:
        # 如果订单结束，记录终点位置，添加订单记录
        order_end_lat = row['lat']
        order_end_lon = row['lon']
        order_end_time = row['time']
        order_end_district = row['region']
        order_end_block = row['block_id']
        orders.append((row['taxi_id'], order_start_lat, order_start_lon, order_end_lat, order_end_lon, order_start_time, order_end_time, order_start_district, order_start_block, order_end_district, order_end_block))
        order_started = False

NameError: name 'df_grouped' is not defined

In [20]:
# 将订单记录整合到DataFrame中
orders_df = pd.DataFrame(orders, columns=['taxi_id', 'start_lat', 'start_lon', 'end_lat', 'end_lon', 'start_time', 'end_time', 'start_district', 'start_block', 'end_district', 'end_block'])
orders_df.to_csv('../res/orders.csv', index=False)

In [53]:
# 各区街道出发、到达数量统计
df_orders = pd.read_csv('../res/orders.csv')

# 根据district_block进行分组，统计每个区块的订单数量
from_district_block = df_orders.groupby('start_block')['taxi_id'].count().sort_values(ascending=False)
to_district_block = df_orders.groupby('end_block')['taxi_id'].count().sort_values(ascending=False)

# print(from_district_block)
from_district_block.to_csv('../res/from.csv')
# print(to_district_block)

import geopandas as gpd
gdf = gpd.read_file('../res/sz_block_mars.json')

grouped_from = df_orders.groupby(['start_block'])['taxi_id'].count()
df_from = pd.DataFrame(grouped_from).reset_index().rename(columns={"start_block": "BLOCK_ID", "taxi_id": "from_count"})

grouped_to = df_orders.groupby(["end_block"])["taxi_id"].count()
df_to = pd.DataFrame(grouped_to).reset_index().rename(columns={"end_block": "BLOCK_ID", "taxi_id": "to_count"})

# print(df_from.columns)
# print(df_to.columns)

df_counts = pd.merge(df_from, df_to, on=["BLOCK_ID"], how="outer")

df_counts["total_count"] = df_counts[["from_count", "to_count"]].sum(axis=1)

df_counts.sort_values('BLOCK_ID').to_csv('../res/count.csv')
# print(df_counts[(df_counts['BLOCK_ID']==-1) & (df_counts['DISTRICT']!='福田区')].shape[0])
df_counts = df_counts[df_counts['BLOCK_ID']!=-1]

# 将计数表格与GeoDataFrame合并，以保留区块的地理信息和其他属性信息
gdf_counts = pd.merge(gdf, df_counts, on=["BLOCK_ID"], how="outer")

# print(len(gdf))
# print(gdf_counts.head(5))

import numpy as np
# 将计数信息写入属性表中
gdf_counts["properties"] = gdf_counts.apply(
    lambda row: {
        "ID": row["ID"],
        "AREA": row["AREA"],
        "ID1": row["ID1"],
        "AREA1": row["AREA1"],
        "NAME": row["NAME"],
        "DISTRICT": row["DISTRICT"],
        "DISTRICT_I": row["DISTRICT_I"],
        "BLOCK_ID": row["BLOCK_ID"],
        "from_count": row["from_count"] if not np.isnan(row['from_count']) else 0,
        "to_count": row["to_count"] if not np.isnan(row['to_count']) else 0,
        "total_count": row["total_count"] if not np.isnan(row['total_count']) else 0
    },
    axis=1
)

# 提取所有区块的GeoDataFrame并保存到新的GeoJSON文件中
gdf_counts[["geometry", "properties"]].to_file("../res/sz_block_mars_with_count.json", driver="GeoJSON")

In [4]:
# 从各区到各区的概率
cross_tab = pd.crosstab(df_orders['start_district'], df_orders['end_district'], normalize='index')
print(cross_tab)

end_district         光明区       南山区       坪山区       宝安区       盐田区       福田区  \
start_district                                                               
光明区             0.528090  0.022472  0.000000  0.314607  0.000000  0.011236   
南山区             0.000963  0.728957  0.000170  0.087232  0.001246  0.126487   
坪山区             0.000000  0.016393  0.409836  0.000000  0.000000  0.000000   
宝安区             0.005139  0.123955  0.000087  0.770122  0.000958  0.053746   
盐田区             0.001739  0.010435  0.000000  0.022609  0.460870  0.109565   
福田区             0.000207  0.075002  0.000089  0.021336  0.003310  0.692781   
罗湖区             0.000144  0.024374  0.000096  0.013169  0.010822  0.240387   
龙华区             0.002390  0.028492  0.000191  0.020748  0.000861  0.071039   
龙岗区             0.000139  0.007041  0.004740  0.008574  0.001464  0.044197   
龙岗区2            0.000000  0.000000  0.333333  0.000000  0.333333  0.000000   

end_district         罗湖区       龙华区       龙岗区      龙岗区2  
start_